# Hackaton

Creación de Estructura de Base de Datos

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient
from tqdm import tqdm_notebook as tqdm
import json


In [11]:
#Login
f= open("k.txt","r")
pw=f.read()
f.close()
connect_local='mongodb://localhost:27017'
connect='mongodb+srv://albertoid:{}@ironhack-hackaton-6ah57.azure.mongodb.net/test?retryWrites=true&w=majority'.format(pw)

try:
    client=MongoClient(connect)
except:
    print('Error de conexion')

In [12]:
client.list_database_names()

['Hackaton', 'admin', 'local']

In [13]:
cHack_Places=client.Hackaton.Places
cHack_Users=client.Hackaton.Users

In [14]:
df_Users=pd.DataFrame()
df_Users=pd.DataFrame(columns=['user_id','email','google_id','password'])
df_Users=df_Users.append({'user_id':'alberto','email':'a@gmail.com','google_id':'a@gmail.com','password':'1234'},ignore_index=True)
df_Users=df_Users.append({'user_id':'karen','email':'k@gmail.com','google_id':'k@gmail.com','password':'qwer'},ignore_index=True)
df_Users=df_Users.append({'user_id':'israel','email':'i@gmail.com','google_id':'i@gmail.com','password':'asdf'},ignore_index=True)
df_Users=df_Users.append({'user_id':'ale','email':'ale@gmail.com','google_id':'ale@gmail.com','password':'zxcv'},ignore_index=True)

In [16]:
Users=df_Users.to_dict('records')


In [216]:
try:
    cHack_Users.insert_many(Users)
except:
    print('No cargó')

In [17]:
#Estructura Básica del Data Frame
df_Places=pd.DataFrame(columns=['name','gps','address','price','kind','cap',
                                'schedule','menu','service','bike','phone',
                                'email','social','reviews'])

In [18]:
df_Users=pd.DataFrame(columns=['user_id','email','google_id','password'])

In [19]:
#Carga de datos iniciales del INEGI
inegi=pd.read_csv('./inegi_cdmx.csv',encoding='ISO-8859-1')

#Busqueda de actividades economicas
actividades=inegi.nombre_act.unique()
af=[]
for i in actividades:
    #i=i.upper()
    #print(i)
    if i.find('Restaurante')>=0:
        af.append(i)
    elif i.find('COMIDA')>=0:
        af.append(i)

#Limpieza de columnas no relevantes
dcol=['id','codigo_act','raz_social','tipo_vial','manzana','fecha_alta','ageb','entidad','cve_mun','cve_loc','tipo_v_e_1','nom_v_e_1','tipo_v_e_2','tipo_v_e_3','nom_v_e_3','nom_v_e_2','cve_ent','localidad','tipoUniEco','edificio','edificio_e','numero_int','letra_int','tipoCenCom','nom_CenCom','num_local']
inegi = inegi.drop(columns=dcol)

#Filtro Restaurantes
inegi = inegi[(inegi.nombre_act==af[0]) | (inegi.nombre_act==af[1]) | (inegi.nombre_act==af[2]) | (inegi.nombre_act==af[3]) | (inegi.nombre_act==af[4]) | (inegi.nombre_act==af[5]) | (inegi.nombre_act==af[6])]

#Filtro Municipio = Cuahutemoc
inegi=inegi[inegi.municipio=='Cuauhtémoc']

inegi=inegi.reset_index()
inegi=inegi.drop(columns='index')
inegi=inegi.drop(columns='nombre_act')
inegi=inegi.fillna('')


/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
inegi.numero_ext=inegi.numero_ext.apply(lambda x: str(x))
inegi.cod_postal=inegi.cod_postal.apply(lambda x: str(x))

In [21]:
inegi['address'] = [{'street':inegi.nom_vial[i], 'number':inegi.numero_ext[i],
                     'letter':inegi.letra_ext[i], 'townshiptype':inegi.tipo_asent[i],
                      'township':inegi.nomb_asent[i], 'zip':inegi.cod_postal[i],
                      'town':inegi.municipio[i]} for i in range(len(inegi))]
inegi['gps'] = [{'latitude':inegi.latitud[i],'logitude':inegi.longitud[i]} for i in range(len(inegi))]
inegi['social'] = [{'web':inegi.www[i]}for i in range(len(inegi))]

In [ ]:
inegi.rename(columns={'nom_estab':'name','per_ocu':'cap','telefono':'phone','correoelec':'email'},inplace=True)
inegi=inegi.drop(columns = ['nom_vial','numero_ext','letra_ext','tipo_asent','nomb_asent','cod_postal',
                            'municipio','latitud','longitud','www'])

imagen='https://www.ironhack.com/assets/icons/ironhack_logos/logo.svg'
inegi['picture']=imagen

In [25]:
inegi.head(1)

,name,cap,phone,email,address,gps,social,picture
0,100% CARNÍVORO,0 a 5 personas,5.51916e+09,,"{'street': 'RIO SENA', 'number': '', 'letter':...","{'latitude': 19.43408487, 'logitude': -99.1680...",{'web': ''},https://www.ironhack.com/assets/icons/ironhack...


In [26]:
data=inegi.to_dict('records')

In [27]:
cHack_Places.insert_many(data)